## Thinkful exercise 3.2.5 - Dimensionality reduction using random forest model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Project description

Data:  2015 data from Lending Club, found at this link:https://www.lendingclub.com/info/download-data.action

Modelling objective: Predict the state of a loan given some information about it

Assignment details: 

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?


### Preprocessing data

In [2]:
#Using nrows to restrict dataset size for development
y2015 = pd.read_csv(
    '../../Datafiles/unit_3/LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    low_memory = False
)
y2015 = y2015.sample(frac=0.20).reset_index(drop=True)

In [3]:
y2015.shape

(84219, 111)

#### The data dictionary includes 152 columns. This is a subset of the columns in the full dataset, apparently.

In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009900,72868653.0,35000.0,35000.0,35000.0,60 months,11.99%,778.38,C,C1,...,0.0,1.0,89.5,36.4,0.0,0.0,307264.0,79260.0,32900.0,48752.0
1,64260389,68730110.0,5000.0,5000.0,5000.0,36 months,8.38%,157.56,B,B1,...,0.0,2.0,100.0,0.0,0.0,0.0,33018.0,17237.0,10100.0,13418.0
2,42263776,45230504.0,14700.0,14700.0,14675.0,36 months,14.65%,507.07,C,C5,...,0.0,2.0,97.0,100.0,0.0,0.0,91886.0,97630.0,17400.0,70486.0
3,55281837,58862656.0,8000.0,8000.0,8000.0,36 months,15.61%,279.72,D,D1,...,0.0,3.0,70.0,100.0,0.0,0.0,28205.0,15150.0,3600.0,14505.0
4,41140327,44016109.0,16000.0,16000.0,16000.0,60 months,14.65%,377.71,C,C5,...,0.0,3.0,100.0,50.0,0.0,0.0,185907.0,66023.0,5000.0,85707.0


In [5]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
84214,39749161,42572876.0,7000.0,7000.0,7000.0,36 months,11.44%,230.64,B,B4,...,0.0,0.0,100.0,0.0,1.0,1.0,33767.0,24074.0,1100.0,27567.0
84215,43380158,46406908.0,20000.0,20000.0,19900.0,36 months,10.99%,654.68,B,B4,...,0.0,0.0,100.0,60.0,0.0,0.0,23600.0,18445.0,19800.0,0.0
84216,40620417,43485131.0,21000.0,21000.0,21000.0,60 months,18.25%,536.13,E,E1,...,0.0,0.0,100.0,50.0,0.0,0.0,42428.0,29024.0,7800.0,17528.0
84217,60035358,63980076.0,18125.0,18125.0,18125.0,36 months,13.33%,613.59,C,C3,...,0.0,0.0,100.0,75.0,0.0,2.0,274782.0,56503.0,6500.0,69782.0
84218,61491159,65609944.0,21475.0,21475.0,21475.0,36 months,12.69%,720.38,C,C2,...,0.0,7.0,97.9,0.0,0.0,0.0,297493.0,70980.0,48700.0,52994.0


In [6]:
#remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [7]:
y2015.shape

(84217, 111)

#### Analyze columns with null values and adjust contents to zero or drop them

In [8]:
#Compute proportion of null values
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                           col_name  null_pcnt
10                        emp_title       5.57
11                       emp_length       5.55
19                             desc      99.98
21                            title       0.02
28           mths_since_last_delinq      48.61
29           mths_since_last_record      82.38
33                       revol_util       0.04
45                     last_pymnt_d       0.07
47                     next_pymnt_d      27.63
50      mths_since_last_major_derog      70.99
53                 annual_inc_joint      99.89
54                        dti_joint      99.89
55        verification_status_joint      99.89
59                      open_acc_6m      95.03
60                       open_il_6m      95.03
61                      open_il_12m      95.03
62                      open_il_24m      95.03
63               mths_since_rcnt_il      95.14
64                     total_bal_il      95.03
65                          il_util      95.65
66           

In [9]:
#Fill nan with zero in some columns, because that is what is implied for these columns given knowledge of the data
#Pandas 'startswith' works differently than Python. Python allows a beg and end argument
#num_tl prefix means 'number of accounts'
col_list = y2015.loc[:, y2015.columns.str.startswith('mths_')].columns
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('num_tl_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('mo_sin_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.contains('bc_')].columns)
for col in col_list:
    y2015[[col]] = y2015[[col]].fillna(value = 0)

In [10]:
#Review remaining columns with null values 
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                     col_name  null_pcnt
10                  emp_title       5.57
11                 emp_length       5.55
19                       desc      99.98
21                      title       0.02
33                 revol_util       0.04
45               last_pymnt_d       0.07
47               next_pymnt_d      27.63
53           annual_inc_joint      99.89
54                  dti_joint      99.89
55  verification_status_joint      99.89
59                open_acc_6m      95.03
60                 open_il_6m      95.03
61                open_il_12m      95.03
62                open_il_24m      95.03
64               total_bal_il      95.03
65                    il_util      95.65
66                open_rv_12m      95.03
67                open_rv_24m      95.03
68                 max_bal_bc      95.03
69                   all_util      95.03
71                     inq_fi      95.03
72                total_cu_tl      95.03
73               inq_last_12m      95.03


In [11]:
# Convert interest rate to numeric.
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [12]:
#Identify columns that are not numeric and examine how variable their values are
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
84217
term
2
grade
7
sub_grade
35
emp_title
32804
emp_length
11
home_ownership
3
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
84217
desc
10
purpose
12
title
15
zip_code
871
addr_state
49
earliest_cr_line
600
revol_util
1099
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
3
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [13]:
# Drop columns with many unique variables
drop_cols_list = ['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'id', 'title', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [14]:
y2015 = y2015.drop(columns = drop_cols_list)

In [15]:
#Drop any remaining columns with a high number of Nulls
y2015 = y2015.dropna(axis = 1)

### Thinkful baseline model

Thinkful commentary:

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

### Robin's modeling

#### Examine the target variable

In [16]:
y2015.shape

(84217, 81)

In [17]:
y2015.loan_status.value_counts()

Current               57536
Fully Paid            17567
Charged Off            5705
Late (31-120 days)     1945
In Grace Period         928
Late (16-30 days)       389
Default                 147
Name: loan_status, dtype: int64

Note that the cut-off where delinquency matters is 16 days and more. In reality, it is likely one month and more than one month that is most meaningful.

#### Build some features with meaningful collections & delinquency information. The recent information is the most important.

Collections data and charge off data: It's most material to know whether there were collections or charge offs in the last 12 months.

Delinquency data: If months since last delinq = 0, means > = 0, < 30 days. So if zero, set no_delinq = True,  else False.

Separate months since last delinq into recent delinquencies -- delinq_last_12_mo = True if >0, <= 12 else false

Then drop cols in sel_cols list

In [18]:
#Change the recent delinquency parameter in number of months
recent_delinq_param = 12
#Use apply to set no_delinq, recent_delinq flags
y2015['no_delinq'] = np.where(y2015.mths_since_last_delinq == 0, True, False)
y2015['recent_delinq'] = np.where(((y2015.mths_since_last_delinq > 0) & (y2015.mths_since_last_delinq <= recent_delinq_param)), True, False)

In [19]:
y2015['no_derog'] = np.where(y2015.mths_since_last_major_derog == 0, True, False)
y2015['recent_derog'] = np.where(((y2015.mths_since_last_major_derog > 0) & (y2015.mths_since_last_major_derog <= 12)), True, False)
y2015['collections_12mo'] = np.where(y2015.collections_12_mths_ex_med == 0, True, False)
y2015['chargeoffs_12mo'] = np.where(((y2015.chargeoff_within_12_mths == 0)), True, False)

In [20]:
# Drop columns used to categorize delinquencies, chargeoffs, derogitories
drop_cols_list = ['mths_since_last_major_derog', 'mths_since_last_delinq', 
                  'collections_12_mths_ex_med', 'chargeoff_within_12_mths',
                  'out_prncp_inv']

In [21]:
y2015 = y2015.drop(columns = drop_cols_list)

In [22]:
assert pd.notnull(y2015).all().all()

#### Step 1. Normalize variance and drop columns with low variance

In [23]:
y2015.shape

(84217, 82)

In [24]:
#Set the target variable and drop it from the main dataset
y = y2015.loan_status
y2015 = y2015.drop('loan_status', axis = 1)

In [25]:
#Make a dataframe containing a subset of the columns - numeric only
num_y2015 = pd.DataFrame(y2015.select_dtypes(include=['float64', 'int64']))

In [26]:
from sklearn.feature_selection import VarianceThreshold 

var_thresh = VarianceThreshold(threshold=0.005)
var_thresh.fit(num_y2015 / num_y2015.mean())

#Here I am using a mask to reduce the features. I could have used fit_transform to do everything in one step
mask = var_thresh.get_support()
reduced_num_y2015 = num_y2015.loc[:, mask] 
print(reduced_num_y2015.shape)

(84217, 65)


17 features were dropped as low-variance (above)

#### Step 2. Encode the categorical variables that remain and merge the dataframe with the reduced set of numeric features

In [27]:
y2015 = pd.get_dummies(y2015)

In [28]:
#Merge the dataframe with reduced number of columns with the dataframe that includes encoded categorical variables
keep_cols = y2015.columns.difference(reduced_num_y2015.columns)
print(keep_cols)
y2015_new = pd.merge(reduced_num_y2015, y2015[keep_cols], left_index=True, right_index=True, how='outer')

Index(['application_type_INDIVIDUAL', 'application_type_JOINT',
       'chargeoffs_12mo', 'collections_12mo', 'grade_A', 'grade_B', 'grade_C',
       'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_MORTGAGE',
       'home_ownership_OWN', 'home_ownership_RENT', 'initial_list_status_f',
       'initial_list_status_w', 'issue_d_Apr-2015', 'issue_d_Aug-2015',
       'issue_d_Dec-2015', 'issue_d_Feb-2015', 'issue_d_Jan-2015',
       'issue_d_Jul-2015', 'issue_d_Jun-2015', 'issue_d_Mar-2015',
       'issue_d_May-2015', 'issue_d_Nov-2015', 'issue_d_Oct-2015',
       'issue_d_Sep-2015', 'no_delinq', 'no_derog', 'policy_code',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',
       'purpose_medical', 'purpose_moving', 'purpose_other',
       'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'pymnt_plan_n', 'recent_delinq', 'recent_derog',
       'ter

In [29]:
y2015_new.shape

(84217, 116)

In [30]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

X = y2015_new

rfc = ensemble.RandomForestClassifier()
rfc.fit(X, y)
cross_val_score(rfc, X, y, cv=10)

array([ 0.95868946,  0.95857075,  0.95952042,  0.95880817,  0.95868946,
        0.95903586,  0.95808099,  0.95902126,  0.95972915,  0.95901152])

#### I've achieved a score above 90%, but how many features are really necessary?

In [31]:
# Calculate the correlation matrix and take the absolute value
corr_matrix = y2015_new.corr().abs()

# Create a True/False mask and apply it
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
y2015_corr = corr_matrix.mask(mask)


# List column names of highly correlated features (r > 0.85). Changing the r value to 0.80 drops the score to around 90,
# so I'll stick with the value 0.85.
to_drop = [c for c in y2015_corr.columns if any(y2015_corr[c] >  0.85)]
# Drop the features in the to_drop list
y2015_reduced = y2015_new.drop(to_drop, axis=1)

print("The reduced dataframe has {} columns.".format(y2015_reduced.shape[1]))

The reduced dataframe has 103 columns.


In [32]:
#These columns will be dropped because they are highly correlated with others
print(to_drop)

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'open_acc', 'total_pymnt', 'total_pymnt_inv', 'recoveries', 'tot_cur_bal', 'num_actv_rev_tl', 'total_bal_ex_mort', 'application_type_INDIVIDUAL', 'initial_list_status_f', 'term_ 36 months']


In [33]:
X = y2015_reduced
rfc.fit(X, y)
cross_val_score(rfc, X, y, cv=10)

array([ 0.95572175,  0.95631529,  0.95702754,  0.95584046,  0.95584046,
        0.95606744,  0.95653723,  0.95664568,  0.95842243,  0.95746703])

In [34]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',    
                                    ascending=False
                                    )

In [35]:
feature_importances['cum_importance'] = feature_importances.cumsum(axis = 0)
print(feature_importances.head(40))

                            importance  cum_importance
out_prncp                     0.340008        0.340008
last_pymnt_amnt               0.249519        0.589526
total_rec_prncp               0.104998        0.694524
collection_recovery_fee       0.035923        0.730447
total_rec_int                 0.031532        0.761979
installment                   0.017468        0.779446
member_id                     0.012264        0.791710
int_rate                      0.008421        0.800131
revol_bal                     0.007471        0.807602
dti                           0.006903        0.814505
avg_cur_bal                   0.006316        0.820821
bc_util                       0.006148        0.826969
mo_sin_old_rev_tl_op          0.006061        0.833030
total_bc_limit                0.006005        0.839036
bc_open_to_buy                0.005994        0.845029
tot_hi_cred_lim               0.005938        0.850967
annual_inc                    0.005793        0.856760
total_rev_

#### Using PCA degrades the accuracy score (from 95% accuracy to about 86%) while using 40 features. PCA doesn't perform better than just taking into account the feature importance measure and establishing a cutoff when the accuracy score reaches an acceptable level.

In [36]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([
('scaler', StandardScaler()),
('reducer', PCA(n_components=40)), 
('classifier', ensemble.RandomForestClassifier())])

pipe.fit(X, y)

print(cross_val_score(pipe, X, y, cv = 10))

[ 0.83689459  0.8370133   0.83416429  0.83523267  0.83926876  0.83887438
  0.83826149  0.84713149  0.83368971  0.8310562 ]


### Discussion

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

My random forest model gets above a 90% accuracy score with 25 specific features; the model achieves ~95% accuracy with about 40 features. None of the generated dummies make the cut, so if this model were to go into production, we could skip that step and work with a smaller dataset from the outset.

The additional features I created also did not contribute to explanatory power in a significant way.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

The most important features are outstanding principal and last payment amount. Together they explain almost 70% of the variability in the dataset.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?

Since outstanding principal and last payment amount account for almost 70% of the variability in the dataset, it  would not be possible to answer the question accurately if one excluded payment amount or outstanding principal.

[Note: Outstanding principal column names are 'out_princp' and 'out_princp_inv'). Payment amount column name is 'installment'.]

One question I am left with is, Why does a value like "member id" have any explanatory power at all? Is it because it may go from low to high values and thereby give some information about how recent the loan is?